In [1]:
import json
import os, sys
from sys import modules
import _jsonnet
from tqdm.notebook import tqdm

from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import random

In [2]:
sys.path.append(os.path.abspath('/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/third_party/wikisql'))
# sys.path.append(os.path.abspath('/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL'))

In [3]:
from ratsql.commands.infer import Inferer
from ratsql.datasets.spider import SpiderItem
from ratsql.utils import registry

In [4]:
import torch

from SpeakQL.Allennlp_models.utils.spider import process_sql, evaluation
from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, Postprocess_rewrite_seq

In [5]:
# del modules['SpeakQL.Allennlp_models.utils.misc_utils']
# del EvaluateSQL, EvaluateSQL_full, Postprocess_rewrite_seq
# from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, Postprocess_rewrite_seq

In [6]:
def Load_Rat_sql(root_dir,
                 exp_config_path,
                 model_dir,
                 checkpoint_step=40000):

    exp_config = json.loads(_jsonnet.evaluate_file(exp_config_path))
    
    model_config_path = os.path.join(root_dir, exp_config["model_config"])
    model_config_args = exp_config.get("model_config_args")
    
    infer_config = json.loads(_jsonnet.evaluate_file(model_config_path, tla_codes={'args': json.dumps(model_config_args)}))

    inferer = Inferer(infer_config)
    inferer.device = torch.device("cpu")
    model = inferer.load_model(model_dir, checkpoint_step)
    dataset = registry.construct('dataset', inferer.config['data']['val'])

    for _, schema in dataset.schemas.items():
        model.preproc.enc_preproc._preprocess_schema(schema)
    
    _ret_dict = {
        'model': model,
        'dataset': dataset,
        'inferer': inferer,
    }
    
    return _ret_dict
    

In [7]:
rat_sql_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                  exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-run.jsonnet',
                                  model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_run/bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                  checkpoint_step=40000)
rat_sql_model_dict.keys()


WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'max_count': 5000, 'min_freq': 4, 'save_path': 'data/spider/nl2code-glove,cv_link=true', 'use_seq_elem_rules': True}, 'encoder_preproc': {'compute_cv_link': True, 'compute_sc_link': True, 'count_tokens_in_word_emb_for_vocab': True, 'db_path': 'data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'max_count': 5000, 'min_freq': 4, 'save_path': 'data/spider/nl2code-glove,cv_link=true', 'word_emb': {'kind': '42B', 'lemmatize': True, 'name': 'glove'}}}


Loading model from /Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_run/bs=20,lr=7.4e-04,end_lr=0e0,att=0/model_checkpoint-00040000


DB connections: 100%|██████████| 166/166 [00:01<00:00, 124.44it/s]


dict_keys(['model', 'dataset', 'inferer'])

In [ ]:
rat_sql_asr_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                      exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-ASR-run.jsonnet',
                                      model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_ASR_run/ASR,bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                      checkpoint_step=40000)
rat_sql_asr_model_dict.keys()


In [ ]:
rat_sql_mixed_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                        exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-mixed-run.jsonnet',
                                        model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_mixed_run/mixed,bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                        checkpoint_step=40000)
rat_sql_mixed_model_dict.keys()

In [8]:
model_dicts = {
    'orig': rat_sql_model_dict,
#     'asr': rat_sql_asr_model_dict,
#     'mixed': rat_sql_mixed_model_dict,
}

In [9]:
def Question(q, db_id, model_dict=model_dicts['orig']):
    model = model_dict['model']
    dataset = model_dict['dataset']
    inferer = model_dict['inferer']
    
    spider_schema = dataset.schemas[db_id]
    data_item = SpiderItem(
        text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
        code=None,
        schema=spider_schema,
        orig_schema=spider_schema.orig,
        orig={"question": q}
    )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        return inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)

In [10]:
Question("how many singers do we have?", "concert_singer")

[{'orig_question': 'how many singers do we have?',
  'model_output': {'_type': 'sql',
   'select': {'_type': 'select',
    'is_distinct': False,
    'aggs': [{'_type': 'agg',
      'agg_id': {'_type': 'Count'},
      'val_unit': {'_type': 'Column',
       'col_unit1': {'_type': 'col_unit',
        'agg_id': {'_type': 'NoneAggOp'},
        'col_id': 0,
        'is_distinct': False}}}]},
   'sql_where': {'_type': 'sql_where'},
   'sql_groupby': {'_type': 'sql_groupby'},
   'sql_orderby': {'_type': 'sql_orderby', 'limit': False},
   'sql_ieu': {'_type': 'sql_ieu'},
   'from': {'_type': 'from',
    'table_units': [{'_type': 'Table', 'table_id': 1}]}},
  'inferred_code': 'SELECT Count(*) FROM singer',
  'score': -0.00011539317730324683}]

In [11]:
Question("display the employee i D and salary of all employees who report to pye um, first name.", "hr_1")

[]

In [ ]:
Question("how many singers do we have?", "concert_singer", model_dict=model_dicts['mixed'])

In [ ]:
Question("display the employee i D and salary of all employees who report to pye um, first name.", "hr_1",
         model_dict=model_dicts['mixed'])

### Evaluate the trained model

In [ ]:
dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [ ]:
with open(dev_path, 'r') as f:
    dev_dataset = json.load(f)
len(dev_dataset)

In [ ]:
d = dev_dataset[0]
d

In [ ]:
question(d['question'], d['db_id'])

In [ ]:
pred_sql_list = []

for d in tqdm(dev_dataset):
    pred = question(d['question'], d['db_id'])[0]
    pred_sql_list.append(pred['inferred_code'])

len(pred_sql_list)

In [ ]:
dev_pred_path = './output/dev_output.txt'

In [ ]:
# with open(dev_pred_path, 'w') as f:
#     for pred in pred_sql_list:
#         f.write(pred + '\n')

### Predict SQL given transcribed dataset JSON files

In [ ]:
DATASET = 'train'
input_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/{0}_asr_amazon.json'.format(DATASET)
output_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/{0}_asr_amazon_RatsqlPredicted.json'.format(DATASET)


In [ ]:
# Paths for human test 
input_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon.json'
output_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon_RatsqlPredicted.json'


In [ ]:
with open(input_dataset_path, 'r') as f:
    asr_dataset = json.load(f)
len(asr_dataset)

In [ ]:
asr_dataset[0].keys()

In [ ]:
for i, d in tqdm(enumerate(asr_dataset), total=len(asr_dataset)):
    if 'ratsql_pred_sql' in d:
        continue
        
    pred = Question(d['question'], d['db_id'])
    if len(pred) == 0:
        print('{}: question({}, {}) failed'.format(i, d['question'], d['db_id']))
        d['ratsql_pred_sql'] = ''
    else:
        d['ratsql_pred_sql'] = pred[0]['inferred_code']

asr_dataset[0]

In [ ]:
d

In [ ]:
pred = question(d['question'], d['db_id'])
pred

In [ ]:
# with open(output_dataset_path, 'w') as f:
#     json.dump(asr_dataset, f, indent=4)

### Predict & Evaluate SQL given rewriter output / original / ASR cands

In [12]:
EvaluateSQL(pred_str='SELECT * FROM singer WHERE name = "Joe Sharp"',
            gold_str='SELECT * FROM singer WHERE name = "DEF"',
            db='concert_singer'), \
EvaluateSQL(pred_str='SELECT country FROM singer WHERE name = "ABC"',
            gold_str='SELECT country FROM singer WHERE name = "DEF" ORDER BY age LIMIT 1',
            db='concert_singer')

((1, 1.0, 0), (0, 0.5, 1))

#### Basic compares

In [16]:
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [ ]:
# human test 
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

In [17]:
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(547, 3075, 1034)

In [19]:
# Just using the 1st ASR candidate, no correction 

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
    _pred_sql = Question(c['question'], _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']
    
    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact
    c['exec'] = _exec

    _question_toks = [_t.lower() for _t in c['question_toks']]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]
    ref_list.append([_gold_question_toks])
    hyp_list.append(_question_toks)


In [37]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
_avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

# ## Std-dev (1st cand only)
# _std_1st = np.std([d[0]['score'] for d in test_dataset])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

# print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')


avg = 0.7274
avg_exact = 0.4552
avg_exec = 0.3327
BLEU = 0.8010


In [35]:
EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset],
                 etype='all')


                     easy                 medium               hard                 extra                all                 
count                136                  240                  91                   80                   547                 
=====================   EXECUTION ACCURACY     =====================
execution            0.397                0.287                0.352                0.338                0.333               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.625                0.446                0.385                0.275                0.455               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               0.846                0.667                0.769                0.738                0.739               
select(no AGG)       0.882                0.671                0.769                0.738                0.750               
where                0.614        

In [36]:
# Dump the predictions 
# Orig
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/Assembly/First-cands.json'
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/Assembly/First-cands.json'
# Mixed
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/Assembly/First-cands.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [38]:
# Using original text (no ASR)

with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(547, 3075, 1034)

In [44]:
# Using original text (no ASR)

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
    _pred_sql = Question(c['gold_question'], _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']
    
    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact
    c['exec'] = _exec


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed



In [45]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
_avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

# ## Std-dev (1st cand only)
# _std_1st = np.std([d[0]['score'] for d in test_dataset])

# print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))


avg = 0.8316
avg_exact = 0.6234
avg_exec = 0.4095


In [46]:
EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset],
                 etype='all')

eval_err_num:1

                     easy                 medium               hard                 extra                all                 
count                136                  240                  91                   80                   547                 
=====================   EXECUTION ACCURACY     =====================
execution            0.463                0.375                0.418                0.412                0.410               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.787                0.658                0.462                0.425                0.623               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               0.926                0.812                0.934                0.838                0.864               
select(no AGG)       0.963                0.820                0.934                0.838                0.877               
where              

In [ ]:
random_subset = random.sample(test_dataset, k=100)

EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in random_subset],
                 glist=[d[0]['query'] for d in random_subset],
                 db_id_list=[d[0]['db_id'] for d in random_subset],
                 etype='all')

#### BRIDGE results 

In [72]:
bridge_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/TabularSemanticParsing/server/spider.bridge.lstm.meta.ts.ppl-0.85.2.dn.eo.feat.bert-base-uncased.xavier-768-400-400-8-4-0.0005-inv-sqr-0.0005-4000-6e-05-inv-sqr-3e-05-4000-0.1-0.3-0.0-0.0-1-8-0.0-0.0-res-0.2-0.0-ff-0.4-0.0.210126-135032.7kf9/predictions.16.ASR.dev.txt'
gold_query_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev_gold.sql'
test_index_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_test.txt'
# db_id_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/db_id.txt'

In [73]:
with open(bridge_pred_path, 'r') as f:
    bridge_preds = f.read().strip().split('\n')
with open(gold_query_path, 'r') as f:
    _lines = [l.split('\t') for l in f.read().strip().split('\n')]
    gold_queries = [l[0] for l in _lines]
    db_ids = [l[1] for l in _lines]
# with open(db_id_path, 'r') as f:
#     db_ids = f.read().strip().split('\n')
with open(test_index_path, 'r') as f:
    test_indices = [int(i) for i in f.read().strip().split('\n')]
    
len(bridge_preds), len(gold_queries), len(db_ids), len(test_indices)

(1034, 1034, 1034, 547)

In [74]:
EvaluateSQL_full(plist=[bridge_preds[idx] for idx in test_indices],
                 glist=[gold_queries[idx] for idx in test_indices],
                 db_id_list=[db_ids[idx] for idx in test_indices],
                 etype='all')

eval_err_num:1
eval_err_num:2
eval_err_num:3

                     easy                 medium               hard                 extra                all                 
count                136                  240                  91                   80                   547                 
=====================   EXECUTION ACCURACY     =====================
execution            0.654                0.442                0.451                0.325                0.479               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.750                0.450                0.429                0.287                0.497               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               0.882                0.679                0.824                0.762                0.767               
select(no AGG)       0.904                0.688                0.824                0.762                0.776     

In [75]:
EvaluateSQL_full(plist=bridge_preds,
                 glist=gold_queries,
                 db_id_list=db_ids,
                 etype='all')

eval_err_num:1
eval_err_num:2
eval_err_num:3
eval_err_num:4

                     easy                 medium               hard                 extra                all                 
count                250                  440                  174                  170                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.652                0.439                0.529                0.276                0.479               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.748                0.468                0.500                0.253                0.506               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               0.871                0.666                0.851                0.612                0.738               
select(no AGG)       0.900                0.670                0.862                0.612           

### End2end

In [27]:
VERSION = '4.0.1.0_laststep'

end2end_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [31]:
with open(end2end_pred_path, 'r') as f:
    end2end_preds = [json.loads(l) for l in f]
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)
    
len(end2end_preds), len(orig_dev_dataset)

(3075, 1034)

In [32]:
# Quick evaluation: only using the 1st ASR candidate

_seen_ids = set()
_first_cand_preds = []

for i, p in tqdm(enumerate(end2end_preds), total=len(end2end_preds)):

    # p = rewriter_ILM_preds[pred_idx]
    # _o_idx = c['original_id']
    # o = orig_dev_dataset[_o_idx]
    # assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    # assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _o_idx = p['original_id']
    if _o_idx in _seen_ids:
        continue
    else:
        _seen_ids.add(_o_idx)
        
    o = orig_dev_dataset[_o_idx]
    
    # Debug 
    # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

    _db_id = o['db_id']
    _pred_sql = p['pred_sql']
    _gold_sql = p['gold_sql']
    assert _gold_sql == o['query'], (_gold_sql, o['query'])
    
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    # Save prediction results 
    p['score'] = _score
    p['exact'] = _exact
    p['exec'] = _exec
    
    _first_cand_preds.append(p)
    

g_str: SELECT first_name ,  last_name FROM players ORDER BY birth_date
p_str: SELECT matches.winner_name, matches.winner_name FROM matches ORDER BY matches.winner_name Asc
g_str: SELECT first_name ,  last_name FROM players ORDER BY birth_date
p_str: SELECT DISTINCT matches.winner_name FROM players JOIN matches ON players.player_id = matches.loser_id ORDER BY players.last_name Asc
SELECT Count(DISTINCT conductor.Conductor_ID) FROM conductor JOIN orchestra ON conductor.Conductor_ID = orchestra.Conductor_ID AND conductor.Conductor_ID = orchestra.Conductor_ID JOIN performance ON orchestra.Orchestra_ID = performance.Orchestra_ID AND orchestra.Orchestra_ID = performance.Orchestra_ID JOIN show ON performance.Performance_ID = show.Performance_ID WHERE show.If_first_show = 'terminal' OR performance.Official_ratings_(millions) = 'terminal'
SELECT COUNT(*) FROM orchestra WHERE Major_Record_Format  =  "CD" OR Major_Record_Format  =  "DVD"
orchestra
process_sql.get_sql() failed



In [33]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([p['score'] for p in _first_cand_preds]) / len(_first_cand_preds)
_avg_exact_1st = sum([p['exact'] for p in _first_cand_preds]) / len(_first_cand_preds)
_avg_exec_1st = sum([p['exec'] for p in _first_cand_preds]) / len(_first_cand_preds)

## Std-dev (1st cand only)
# _std_1st = np.std([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']])

print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))


avg = 0.3425
avg_exact = 0.0146
avg_exec = 0.0256


#### Reranker

In [91]:
VERSION = '1.14.1.2'
HUMAN_TEST = False

if not HUMAN_TEST:
    reranker_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    reranker_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_reranker.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'


In [ ]:
## Aggreg with rewriter cands 

RERANKER_VERSION = '1.10.0.2'
REWRITER_VERSION = '2.6.0.2t-2.6.0.2i'
HUMAN_TEST = True

if not HUMAN_TEST:
    reranker_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/aggreg/output-{RERANKER_VERSION}-with-{REWRITER_VERSION}.json'
    test_dataset_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/aggreg_extra_cands/test_reranker_with_{REWRITER_VERSION}.json'
    orig_dev_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    reranker_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/aggreg/output-humantest-yshao-{RERANKER_VERSION}-with-{REWRITER_VERSION}.json'
    test_dataset_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/aggreg_extra_cands/human_test_yshao_reranker_with_{REWRITER_VERSION}.json'
    orig_dev_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'
    

In [92]:
with open(reranker_pred_path, 'r') as f:
    reranker_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(reranker_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(3075, 547, 3075, 1034)

In [93]:
reranker_preds[0].keys(), test_dataset[0][0].keys()

(dict_keys(['score_preds', 'question', 'original_id']),
 dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'span_ranges', 'original_id', 'ratsql_pred_sql', 'gold_question', 'gold_question_toks', 'ratsql_pred_exact', 'ratsql_pred_score', 'question_toks_edit_distance']))

In [94]:
pred_idx = 0

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
    
    # Find the cand with highest score 
    d_preds = []
    for _ in d:
        p = reranker_preds[pred_idx]
        d_preds.append(p)
        pred_idx += 1
    
    _c_idx = np.argmax([p['score_preds'] for p in d_preds])
    
    c = d[_c_idx]
    p = d_preds[_c_idx]
    
    for _c in d:
        _c['is_reranker_selection'] = False
    c['is_reranker_selection'] = True
    
    # Use the selected cand to proceed 
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']

    _question = c['question']

    if c['ratsql_pred_sql'] is not None:
        # If SQL is already predicted, no need to predict again 
        _pred_sql = c['ratsql_pred_sql']
    else:
        # If SQL not yet predicted, do it 
        _pred_sql = Question(_question, _db_id)[0]['inferred_code']

    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

#     # Save the taggerILM raw outputs, for later aggregation 
#     c['pred_tags'] = p['rewriter_tags']
#     c['pred_ILM'] = p['rewrite_seq_prediction']
#     c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']
    
    # Save prediction results 
    # c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    # p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score
    c['exact'] = p['exact'] = _exact
    c['exec'] = p['exec'] = _exec

    _question_toks = [_t.lower() for _t in c['question_toks']]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

    ref_list.append([_gold_question_toks])
    hyp_list.append(_question_toks)


In [95]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)
_avg_exact_1st = sum([c['exact'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)
_avg_exec_1st = sum([c['exec'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')



avg = 0.7599 (std = 0.2977)
avg_exact = 0.5027
avg_exec = 0.3620
BLEU = 0.7853


In [ ]:
EvaluateSQL_full(plist=[c['pred_sql'] for d in test_dataset for c in d if c['is_reranker_selection']],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset])

In [96]:
# Non ASR
if not HUMAN_TEST:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
else:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
    
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'

with open(test_output_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)

#### Tagger-ILM

In [71]:
## Tagger accuracy 
VERSION = '2.12.7.2t'

predict_fname = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'


In [72]:
with open(predict_fname, 'r') as f:
    predicts = [json.loads(l) for l in f]
len(predicts)

3075

In [73]:
correct_tags = 0
total_tags = 0

non_O_matches = 0
non_O_preds = 0
non_O_golds = 0

for p in predicts:
    _pred_tags = p['tags_prediction']
    _gold_tags = [t for t in p['gold_tags'] if t != 'O']
    assert len(_pred_tags) == len(_gold_tags)
    
    _cor = sum([_p == _g for _p, _g in zip(_pred_tags, _gold_tags)])
    _total = len(_pred_tags)
    assert -1e-8 < _cor / _total - p['tags_accuracy'] < 1e-8, f"{_cor / _total}, {p['tags_accuracy']}"
    
    for _p, _g in zip(_pred_tags, _gold_tags):
        if _p != 'O-KEEP':
            non_O_preds += 1
        if _g != 'O-KEEP':
            non_O_golds += 1
        if _p == _g and _p != 'O-KEEP':
            non_O_matches += 1
    
    correct_tags += _cor
    total_tags += _total

tagger_prec = non_O_matches / non_O_preds
tagger_recall = non_O_matches / non_O_golds
tagger_F1 = 2 * tagger_prec * tagger_recall / (tagger_prec + tagger_recall)
    
print(f'Overall tagger accuracy = {correct_tags}/{total_tags} = {correct_tags / total_tags:.4f}')
print(f'Tagger P = {tagger_prec:.4f}, R = {tagger_recall:.4f}, F1 = {tagger_F1:.4f}')

Overall tagger accuracy = 38648/41776 = 0.9251
Tagger P = 0.7118, R = 0.7006, F1 = 0.7062


In [74]:
## Full tagger-ILM

VERSION = '2.12.7.2t-2.12.7.2i'
HUMAN_TEST = False
ASR = 'Amazon'

if not HUMAN_TEST:
    if ASR == 'AssemblyAI':
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'



In [75]:
with open(rewriter_ILM_pred_path, 'r') as f:
    rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(3075, 547, 3075, 1034)

In [76]:
# Quick evaluation: only using the 1st ASR candidate

pred_idx = 0

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
    
    p = rewriter_ILM_preds[pred_idx]
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    # Debug 
    # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

    _db_id = o['db_id']

    # _tags = p['tags_prediction']  # For previous taggerILM joint model 
    # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')
    
    _tags = p['rewriter_tags']
    _rewrite_seq = p['rewrite_seq_prediction']
    _question_toks = c['question_toks']

    _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
    _rewritten_question = ' '.join(_rewritten_question_toks)

    _pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']

    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

    # Save the taggerILM raw outputs, for later aggregation 
    c['pred_tags'] = p['rewriter_tags']
    c['pred_ILM'] = p['rewrite_seq_prediction']
    c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']
    
    # Save prediction results 
    c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score
    c['exact'] = p['exact'] = _exact
    c['exec'] = p['exec'] = _exec

    _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

    ref_list.append([_gold_question_toks])
    hyp_list.append(_rewritten_question_toks)
    
    pred_idx += len(d)

    

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['and'], ['named', '``'], ['chervil', "''"]]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['airports', "'"], ['?']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP

In [77]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
_avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')


avg = 0.7535 (std = 0.2996)
avg_exact = 0.4936
avg_exec = 0.3601
BLEU = 0.8658


In [78]:
# Non ASR
if not HUMAN_TEST:
    if ASR == 'AssemblyAI':
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
else:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'
# Mixed
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/{VERSION}.json'

with open(test_output_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)

#### Loading from predicted file (only 1st cand is predicted!)

In [ ]:
VERSION = '3.6.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
# Using EvaluateSQL_full 

# tables_json = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/tables.json'
# kmaps = evaluation.build_foreign_key_map_from_json(tables_json)

plist = [d[0]['pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)


In [ ]:
# For reranker output file 
VERSION = '1.10.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
plist = [c['ratsql_pred_sql'] for d in test_dataset for c in d if c['is_reranker_selection'] == 1]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
plist = [d[0]['ratsql_pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
# Using all ASR candidates (no longer in use)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        # _tags = p['tags_prediction']
        _tags = p['tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Evaluation process with oracle tags (no longer in use for version>=2.3.0)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        _tags = p['gold_tags']
        _rewrite_seq = p['oracle_tags_rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question'] = _rewritten_question
        c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql'] = _pred_sql
        c['oracle_tags_score'] = p['oracle_tags_score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['oracle_tags_score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['oracle_tags_score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_oracle_avg_1st = sum([d[0]['oracle_tags_score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_oracle_std_1st = np.std([d[0]['oracle_tags_score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_oracle_avg_1st, _oracle_std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['tag_prediction'] = list(zip(p['question'].split(' '), p['tags_prediction']))
        _pred_c['rewrite_seq'] = []
        for t in p['rewrite_seq_prediction']:
            _pred_c['rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['rewritten_question'] = p['rewritten_question']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_tags'] = list(zip(p['question'].split(' '), p['gold_tags']))
        _pred_c['oracle_tags_rewrite_seq'] = []
        for t in p['oracle_tags_rewrite_seq_prediction']:
            _pred_c['oracle_tags_rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question']
        _pred_c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql']
        _pred_c['oracle_tags_score'] = p['oracle_tags_score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
with open('./output/test-prediction-{}.json'.format(VERSION), 'w') as f:
    json.dump(test_pred_dataset, f, indent=4)

In [ ]:
# Load dataset file with predictions 

with open('./output/pred-{}.json'.format(VERSION), 'r') as f:
    test_pred_dataset = json.load(f)
len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
test_pred_dataset[0][0].keys()

In [ ]:
# Analysis 
orig_dev_preds_path = './output/dev_output.txt'

with open(orig_dev_preds_path, 'r') as f:
    orig_dev_preds = [l.strip() for l in f.readlines()]

len(orig_dev_preds)

In [ ]:
# for d in test_dataset[5::40]:
#     print('DB:', d[0]['db_id'])
#     print('ASR question:\t\t', d[0]['question'])
#     print('Rewritten question:\t', d[0]['rewritten_question'])
#     print('Gold question:\t\t', d[0]['gold_question'])
#     print('ASR-q Pred SQL:\t\t', d[0]['ratsql_pred_sql'])
#     print('Rewritten-q Pred SQL:\t', d[0]['pred_sql'])
#     print('Gold-q Pred SQL:\t', orig_dev_preds[d[0]['original_id']])
#     print('Gold SQL:\t\t', d[0]['query'])
#     print('Score:', d[0]['score'])
#     print('-'*50)

In [ ]:
test_pred_samples = [c for d in test_pred_dataset for c in d]
for i, c in list(enumerate(test_pred_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['ASR_question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
    print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['gold_sql'])
    print('Score:', c['score'])
    

In [ ]:
inspect_ids = [8, 96, 272, 448, 1416, 1592, 1680, 1856, 2120, 2296, 2384, 2560, 2824]

#### Seq2seq

In [34]:
## Batch evaluating 

VERSION_LIST = ['3.12.1.1', '3.12.1.2']
HUMAN_TEST = False
ASR = 'Amazon'

for VERSION in VERSION_LIST:
    print(f'Evaluating ({ASR}){" (human test)" if HUMAN_TEST else ""}: VERSION = {VERSION}')
    
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{VERSION}.json'
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        # human test 
        rewriter_s2s_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    
    with open(rewriter_s2s_pred_path, 'r') as f:
        rewriter_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)
        
    # Quick evaluation: only using the 1st ASR candidate

    ref_list = []
    hyp_list = []
    
    pred_idx = 0

    for d in tqdm(test_dataset):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        _db_id = o['db_id']

        # _tags = p['tags']
        # _rewrite_seq = p['rewrite_seq_prediction']
        # _question_toks = c['question_toks']

        # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        # _rewritten_question = ' '.join(_rewritten_question_toks)

        _rewritten_question = ' '.join(p['s2s_prediction'])

        if _rewritten_question == '':
            print(f'_rewritten_question is empty')
            _pred_sql = ''
            _gold_sql = c['query']
            _exact = _score = _exec = 0
        else:
            _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
            _gold_sql = c['query']
            _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec
        
        # For BLEU 
        _rewritten_question_toks = [_t.lower() for _t in p['s2s_prediction']]
        _question_toks = [_t.lower() for _t in c['question_toks']]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])
    
    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)
    
    print(f'VERSION {VERSION}:')
    print(f'avg = {_avg_1st:.4f} (std = {_std_1st:.4f})')
    print(f'avg_exact = {_avg_exact_1st:.4f}')
    print(f'avg_exec = {_avg_exec_1st:.4f}')
    print(f'BLEU = {_bleu:.4f}')
    print()
    
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
        
    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)


Evaluating (Amazon): VERSION = 3.12.1.1



VERSION 3.12.1.1:
avg = 0.7410 (std = 0.3078)
avg_exact = 0.4826
avg_exec = 0.3638
BLEU = 0.8532

Evaluating (Amazon): VERSION = 3.12.1.2


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

VERSION 3.12.1.2:
avg = 0.7478 (std = 0.3037)
avg_exact = 0.4899
avg_exec = 0.3675
BLEU = 0.8566



In [ ]:
# Single version, not used 

VERSION = '3.3.0.0'

rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [ ]:
with open(rewriter_s2s_pred_path, 'r') as f:
    rewriter_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(rewriter_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

In [ ]:
rewriter_preds[0].keys()

In [ ]:
for p in rewriter_preds[3::300]:
    print(p['question'])
    print(' '.join(p['s2s_prediction']))
    print(' '.join(p['gold_rewrite_seq_s2s'][1:-1]))
    print('-'*50)

In [ ]:
# Quick evaluation: only using the 1st ASR candidate

pred_idx = 0

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
    
    p = rewriter_preds[pred_idx]
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

    _db_id = o['db_id']

    # _tags = p['tags']
    # _rewrite_seq = p['rewrite_seq_prediction']
    # _question_toks = c['question_toks']

    # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
    # _rewritten_question = ' '.join(_rewritten_question_toks)
    
    _rewritten_question = ' '.join(p['s2s_prediction'])
    
    if _rewritten_question == '':
        print(f'_rewritten_question is empty')
        _pred_sql = ''
        _gold_sql = c['query']
        _score = 0
    else:
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

    c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score

    pred_idx += len(d)

    

In [ ]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
# Actual (full) evaluation process 

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        pred_idx += 1
        if 'score' in c:
            continue  # already inferred  
        
        _db_id = o['db_id']

        _rewritten_question = ' '.join(p['s2s_prediction'])
        _pred_result = Question(_rewritten_question, _db_id)
        
        _gold_sql = c['query']
        
        if len(_pred_result) == 0:
            print(_db_id, _rewritten_question, '-- no predictiction')
            _pred_sql = ''
            _score = 0
        else:
            _pred_sql = _pred_result[0]['inferred_code']
            _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        


In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_preds]) / len(rewriter_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['rewritten_question'] = p['s2s_prediction']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
test_samples = [c for d in test_dataset for c in d]
for i, c in list(enumerate(test_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
#     print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['query'])
    print('Score:', c['score'])
    

### Temp

In [ ]:
rewriter_ILM_preds[0].keys()

In [ ]:
test_dataset[0][0].keys()

In [ ]:
orig_dev_dataset[0].keys()

In [ ]:
# Rewrite_seq postprocessing, to get the rewritten question 

_idx = 154

_tags = rewriter_ILM_preds[_idx]['tags_prediction']
_rewrite_seq = rewriter_ILM_preds[_idx]['rewrite_seq_prediction']
_question_toks = rewriter_ILM_preds[_idx]['question'].split(' ')
_tags, _rewrite_seq, _question_toks

postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)

In [ ]:
db = 'concert_singer'
g_str = 'SELECT count(*) FROM singer'
p_str = "SELECT Count(DISTINCT singer.Name) FROM singer WHERE singer.Name = 'terminal'"

db, p_str, g_str, EvaluateSQL(p_str, g_str, db)

In [107]:
with open('/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/train/train_reranker.json') as f:
    _train = json.load(f)
len(_train), sum([len(d) for d in _train])

(7000, 41112)

In [108]:
del _train

In [109]:
%load_ext memory_profiler

In [110]:
%memit

peak memory: 623.44 MiB, increment: -2.01 MiB
